# Setup notebook for prometheus service

Create prometheus config for prometheus service

In [1]:
%%writefile ./prometheus.yml
global:
  scrape_interval: 15s

scrape_configs:
  - job_name: 'traefik'
    static_configs:
      - targets: ['traefik:8080']

  - job_name: 'nginx'
    static_configs:
      - targets: ['nginx:80']

  - job_name: 'mongo_manager_db_service'
    static_configs:
      - targets: ['mongo_manager_db_service:27017']

  - job_name: 'redis'
    static_configs:
      - targets: ['redis:6379']


Overwriting ./prometheus.yml


Create prometheus.yml

In [9]:
%%writefile ../prometheus_service.yml

version: '3'


services:
    prometheus:
        image: prom/prometheus:latest
        container_name: prometheus
        
        volumes:
            - ./Prometheus/prometheus.yml:/etc/prometheus/prometheus.yml
            - prometheus-data:/prometheus

        ports:
            - "10000:9090"
        
        networks:
            - app-network

        depends_on:
            nginx:
                condition: service_healthy

        command:
            - '--config.file=/etc/prometheus/prometheus.yml'

volumes:
  prometheus-data: {}

Overwriting ../prometheus_service.yml
